In [1]:
import sys
from os import listdir, path
import MeCab
import re
from gensim import models
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
#聖書よむ
with open("bibletext.txt", "rU") as f:
    text = [v.rstrip() for v in f.readlines()]
    
#先頭が空欄だったので削除
text.pop(0)

#いらない先頭の数字を削除
text_no_numbers=[]
for i in range(0,len(text)):
    text_no_number=re.sub("^\d+.\d+\s", "",text[i])
    text_no_numbers.append(text_no_number)

In [23]:
text_no_numbers[0]

'イエス・キリストの使徒ペトロから，ポントス，ガラテア，カッパドキア，アジア，ビティニアの離散のうちに仮住まいしている選ばれた人たち，'

In [ ]:
#形態素解析。名詞、動詞とかだけ残して、単語をカンマで区切る処理。
tagger = MeCab.Tagger('-Ochasen')
tagger.parse('') # <= 空文字列をparseする
tagger.parseToNode('')

word_classes = [u'名詞',u'動詞',u'形容詞',u'形容動詞',u'副詞']#使いたい文字を指定
keywords = []

for i in range(0,len(text_no_numbers)):
    keyword=[]
    node = tagger.parseToNode(text_no_numbers[i])

    while node:
        if node.feature.split(",")[0] in word_classes:
            keyword.append(node.surface)
        node = node.next
    keywords.append(keyword)

In [ ]:
#１文書ずつ、単語に分割してリストに入れていく[([単語1,単語2,単語3],文書id),...]こんなイメージ
#words：文書に含まれる単語のリスト（単語の重複あり）
#tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）
trainings = [TaggedDocument(words =data, tags = [i]) for (i,data) in enumerate(keywords)]

#トレーニング（パラメータ、これでいいのかな）
m = Doc2Vec(documents= trainings, dm = 1, size=300, window=8, min_count=10, workers=4)

#モデルのセーブ
m.save("doc2vec.model")


In [5]:
trainings[0]

TaggedDocument(words=['イエス', 'キリスト', '使徒', 'ペ', 'トロ', 'ポントス', 'ガラテア', 'カッパドキア', 'アジア', 'ビティニア', '離散', 'うち', '仮住まい', 'し', 'いる', '選ば', 'れ', '人', 'たち'], tags=[0])

In [10]:
# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('doc2vec.model')

In [11]:
#質問文と一番近い聖書のテキストIDを出す。

def search_similar_texts(question):
    x = m.infer_vector(question)
    most_similar_texts = m.docvecs.most_similar([x])
    for similar_text in most_similar_texts:
        return(similar_text[0])

In [12]:
question = ["隣人愛", "とは" ,"何" ]

sim_id=search_similar_texts(question)
print(sim_id,text_no_numbers[sim_id])

31 ですから，信じるあなた方にとっては尊いものですが，不従順な者たちにとってはこのようです。 「建てる者たちが退けた石， それが隅の親石となった」，


In [13]:
question = ["愛", "とは" ,"何" ]

sim_id=search_similar_texts(question)
print(sim_id,text_no_numbers[sim_id])

91 あなた方の中の長老たちに，仲間の長老であり，キリストの苦しみの証人であり，やがて現わされる栄光にあずかる者として，わたしは勧めます。


In [15]:
question = ["将来", "心配" ]

sim_id=search_similar_texts(question)
print(sim_id,text_no_numbers[sim_id])

14 あなた方を召された方の聖なるさまに従って，あなた方も自分のすべての振る舞いにおいて聖なるものとなりなさい。


In [21]:
question = ["お金", "ほしい"]

sim_id=search_similar_texts(question)
print(sim_id,text_no_numbers[sim_id])

72 それで，キリストは肉においてわたしたちのために苦しみを受けられたのですから，あなた方も同じ思いで武装しなさい。肉において苦しみを受けた人は，罪をやめているからです。


In [22]:
question = ["上司", "嫌い" ,"腹立たしい" ]

sim_id=search_similar_texts(question)
print(sim_id,text_no_numbers[sim_id])

96 ですから，神の強力なみ手のもとで自分を低くしなさい。その方がやがてあなた方を高くしてくださるためです。


以下よりツイッターの部

In [130]:
import tweepy
from requests_oauthlib import OAuth1Session
import json

CK = ''                             # Consumer Key
CS = ''         # Consumer Secret
auth = tweepy.OAuthHandler(CK, CS)
AT = '' # Access Token
AS = ''# Accesss Token Secert
auth.set_access_token(AT, AS)

api = tweepy.API(auth)


In [119]:
# ツイート投稿用のURL
url = "https://api.twitter.com/1.1/statuses/update.json"

# ツイート本文
params = {"status": "テスト"}

# OAuth認証で POST method で投稿
twitter = OAuth1Session(CK, CS, AT, AS)
req = twitter.post(url, params = params)

# レスポンスを確認
if req.status_code == 200:
    print ("OK")
else:
    print ("Error: %d" % req.status_code)

OK


In [ ]:
#自動でつぶやく

In [134]:
#返信につぶやく
#参考ページ：http://net.univ-q.com/archives/38

if str(status.in_reply_to_screen_name)=="{Bot_TWITTER_ID}":
    tweet = "@" + str(status.user.screen_name) + "answer" 
    api.update_status(status=tweet)


NameError: name 'status' is not defined

timeline=api.mentions_timeline()
for status in timeline:
　　　status_id=status.id
　　　screen_name=status.author.screen_name.encode("UTF-8")
　　　reply_text="@"+screen_name+" "+"リプライテキスト"
　　　api.update_status(status=reply_text,in_reply_to_status_id=status_id)